In [0]:
import os
import re
import numpy as np
import pandas as pd

In [0]:
df_inflation = pd.read_csv('IFS_03-14-2020 16-11-47-72_timeSeries.csv')
df_gdp = pd.read_csv('IFS_04-07-2020 12-20-28-26_timeSeries.csv')

In [0]:
df_inflation.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Attribute,2000M1,2000M2,2000M3,2000M4,2000M5,2000M6,2000M7,2000M8,2000M9,2000M10,2000M11,2000M12,2001M1,2001M2,2001M3,2001M4,2001M5,2001M6,2001M7,2001M8,2001M9,2001M10,2001M11,2001M12,2002M1,2002M2,2002M3,2002M4,2002M5,2002M6,2002M7,2002M8,2002M9,2002M10,2002M11,...,2017M1,2017M2,2017M3,2017M4,2017M5,2017M6,2017M7,2017M8,2017M9,2017M10,2017M11,2017M12,2018M1,2018M2,2018M3,2018M4,2018M5,2018M6,2018M7,2018M8,2018M9,2018M10,2018M11,2018M12,2019M1,2019M2,2019M3,2019M4,2019M5,2019M6,2019M7,2019M8,2019M9,2019M10,2019M11,2019M12,2020M1,2020M2,Base Year,Unnamed: 248
0,"China, P.R.: Mainland",924,"Prices, Consumer Price Index, All items, Perce...",PCPI_PC_PP_PT,Value,1.320806,1.301178,-1.388877,-0.588980,-0.597836,-0.907688,-0.598034,0.595542,2.011006,-0.000283,0.300488,0.099999,1.021334,0.099989,-0.599990,0.200005,-0.500006,-1.199986,-0.500009,0.099990,0.900010,0.299996,-0.200001,0.100003,0.299991,1.100008,-1.300002,-0.299990,-0.300002,-0.900003,-0.700002,0.299996,1.000000,0.200001,-0.100007,...,1.072125,-0.192864,-0.289855,0.000000,0.096899,-0.193611,-0.096993,0.485437,0.579710,0.096061,0.000000,0.287908,0.669856,1.235741,-1.126761,-0.189934,-0.190295,0.000000,0.285987,0.665399,0.661001,0.187617,-0.374532,0.093985,0.469484,1.028037,-0.370028,0.092851,0.000000,-0.092764,0.371402,0.740056,0.826446,0.910747,0.361011,0.000000,1.438849,NaN,NaN,NaN
1,"Belarus, Rep. of",913,"Prices, Consumer Price Index, All items, Perce...",PCPI_PC_PP_PT,Value,14.100000,9.290096,5.773857,5.079606,4.617605,6.137931,4.678363,3.600248,6.710605,5.165637,5.392419,5.116515,4.800000,3.816794,3.952206,3.271441,2.654110,2.085071,1.633987,0.723473,2.075020,3.596560,4.528302,5.487365,6.100000,3.581527,2.547771,2.928128,2.241379,1.349073,1.164725,1.151316,1.138211,1.688103,3.241107,...,0.870000,0.535343,0.276107,0.658865,0.283314,0.711154,-0.106404,-0.794035,0.253782,1.197546,0.452184,0.172397,0.750000,0.913151,0.777024,0.292797,-0.282211,0.429394,-0.155476,0.126521,0.806765,0.511040,0.556408,0.782293,0.880000,1.338224,0.381493,0.038979,0.331190,-0.058252,0.194288,-0.232693,0.456754,0.541743,0.230925,0.537583,0.860000,NaN,NaN,NaN
2,"Tajikistan, Rep. of",923,"Prices, Consumer Price Index, All items, Perce...",PCPI_PC_PP_PT,Value,NaN,0.800000,2.400000,3.300000,5.900000,1.700000,0.200000,5.900000,7.500000,12.400000,7.700000,-0.300000,2.400000,1.200000,1.300000,2.100000,2.000000,-0.800000,-0.200000,-0.900000,-0.300000,1.000000,1.800000,2.300000,1.600000,0.900000,0.400000,3.100000,0.400000,-0.700000,0.800000,0.500000,2.900000,0.400000,0.600000,...,0.600000,1.124764,1.500188,1.200000,0.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Azerbaijan, Rep. of",912,"Prices, Consumer Price Index, All items, Perce...",PCPI_PC_PP_PT,Value,0.730308,1.027901,0.058279,0.015539,-0.480645,-0.834955,-0.525449,-0.188417,0.735146,0.575450,0.437115,0.649634,0.327930,0.577859,0.277551,0.192262,-0.532763,-0.375475,-0.833657,-0.528164,0.483648,0.565807,0.324510,0.878021,0.639031,0.443672,0.396099,0.945696,0.489083,-1.018483,-0.424372,-0.552050,0.450263,0.614845,0.727277,...,2.320857,1.622746,1.594970,0.238658,0.171032,-1.035063,0.111020,0.199216,0.729418,1.102993,0.181123,0.092961,0.014515,0.154629,0.206745,-0.204086,-0.032774,-0.658192,-0.591018,0.863519,0.248491,0.163502,0.581491,0.785682,0.190598,0.563878,0.606804,0.383925,-0.430774,-0.467068,0.054304,-0.319731,0.346839,0.416770,0.463842,0.536374,0.566053,NaN,NaN,NaN
4,Vietnam,582,"Prices, Consumer Price Index, All items, Perce...",PCPI_PC_PP_PT,Value,0.415628,1.572848,-1.140994,-0.741962,-0.581395,-0.501253,-0.587741,0.084459,-0.168776,0.084531,0.929054,0.083682,0.250836,0.417014,-0.747508,-0.502092,-0.168209,0.000000,-0.168492,0.000000,0.506329,-0.167926,0.252313,0.838926,1.106530,2.241594,-0.852619,0.000000,0.368550,0.122399,-0.122249,0.122399,0.122249,0.366300,0.243309,...,0.456400,0

In [0]:
df_gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Attribute,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Base Year,Unnamed: 25
0,"North Macedonia, Republic of",962,"Net Primary Income from Abroad, Nominal, Domes...",NGDPNPI_XDC,Value,-4.611992e+09,-1.850745e+09,-2.945402e+09,-3.258186e+09,-1.744191e+09,-5.382675e+09,-1.341598e+09,-1.740887e+10,-5.015379e+09,-2.948725e+09,-5.782850e+09,-8.701471e+09,-8.416607e+09,-1.303184e+10,-9.866119e+09,NaN,NaN,NaN,NaN,NaN,NaN
1,Papua New Guinea,853,"Net Primary Income from Abroad, Nominal, Domes...",NGDPNPI_XDC,Value,-3.914000e+08,-4.081000e+08,-4.149000e+08,-4.244000e+08,-4.351000e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,India,534,"Net Primary Income from Abroad, Nominal, Domes...",NGDPNPI_XDC,Value,-2.273300e+11,-2.006800e+11,-1.669000e+11,-2.070800e+11,-2.237500e+11,-2.611600e+11,-3.323400e+11,-2.051200e+11,-3.292300e+11,-3.800000e+11,-8.180700e+11,-7.682400e+11,-1.167660e+12,-1.398840e+12,-1.474300e+12,-1.597800e+12,-1.896000e+12,NaN,NaN,NaN,NaN
3,Lebanon,446,"Net Primary Income from Abroad, Nominal, Domes...",NGDPNPI_XDC,Value,7.390000e+11,1.780000e+11,-3.860000e+11,-6.810000e+11,-5.310000e+11,7.870000e+11,5.670000e+11,6.360000e+11,4.060000e+11,1.287000e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Tanzania, United Rep. of",738,"Net Primary Income from Abroad, Nominal, Domes...",NGDPNPI_XDC,Value,-6.669900e+10,-3.893900e+10,-1.383173e+12,-3.045726e+12,-4.910258e+12,-1.800000e+08,3.637930e+10,6.475378e+10,3.376268e+10,9.819843e+10,1.064698e+11,1.319467e+11,-1.580834e+10,-1.375702e+11,-1.975099e+11,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
new_df_inflation = df_inflation.drop(columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Attribute', 'Base Year', 'Unnamed: 248'])
new_df_gdp = df_gdp.drop(columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Attribute', 'Base Year', 'Unnamed: 25'])

In [0]:
new_df_inflation.head()

,2000M1,2000M2,2000M3,2000M4,2000M5,2000M6,2000M7,2000M8,2000M9,2000M10,2000M11,2000M12,2001M1,2001M2,2001M3,2001M4,2001M5,2001M6,2001M7,2001M8,2001M9,2001M10,2001M11,2001M12,2002M1,2002M2,2002M3,2002M4,2002M5,2002M6,2002M7,2002M8,2002M9,2002M10,2002M11,2002M12,2003M1,2003M2,2003M3,2003M4,...,2016M11,2016M12,2017M1,2017M2,2017M3,2017M4,2017M5,2017M6,2017M7,2017M8,2017M9,2017M10,2017M11,2017M12,2018M1,2018M2,2018M3,2018M4,2018M5,2018M6,2018M7,2018M8,2018M9,2018M10,2018M11,2018M12,2019M1,2019M2,2019M3,2019M4,2019M5,2019M6,2019M7,2019M8,2019M9,2019M10,2019M11,2019M12,2020M1,2020M2
0,1.320806,1.301178,-1.388877,-0.588980,-0.597836,-0.907688,-0.598034,0.595542,2.011006,-0.000283,0.300488,0.099999,1.021334,0.099989,-0.599990,0.200005,-0.500006,-1.199986,-0.500009,0.099990,0.900010,0.299996,-0.200001,0.100003,0.299991,1.100008,-1.300002,-0.299990,-0.300002,-0.900003,-0.700002,0.299996,1.000000,0.200001,-0.100007,0.300008,1.099995,0.900005,-0.600011,-0.199995,...,0.097752,0.195312,1.072125,-0.192864,-0.289855,0.000000,0.096899,-0.193611,-0.096993,0.485437,0.579710,0.096061,0.000000,0.287908,0.669856,1.235741,-1.126761,-0.189934,-0.190295,0.000000,0.285987,0.665399,0.661001,0.187617,-0.374532,0.093985,0.469484,1.028037,-0.370028,0.092851,0.000000,-0.092764,0.371402,0.740056,0.826446,0.910747,0.361011,0.000000,1.438849,NaN
1,14.100000,9.290096,5.773857,5.079606,4.617605,6.137931,4.678363,3.600248,6.710605,5.165637,5.392419,5.116515,4.800000,3.816794,3.952206,3.271441,2.654110,2.085071,1.633987,0.723473,2.075020,3.596560,4.528302,5.487365,6.100000,3.581527,2.547771,2.928128,2.241379,1.349073,1.164725,1.151316,1.138211,1.688103,3.241107,3.215926,4.300000,1.821668,1.789077,2.035153,...,0.787618,0.481599,0.870000,0.535343,0.276107,0.658865,0.283314,0.711154,-0.106404,-0.794035,0.253782,1.197546,0.452184,0.172397,0.750000,0.913151,0.777024,0.292797,-0.282211,0.429394,-0.155476,0.126521,0.806765,0.511040,0.556408,0.782293,0.880000,1.338224,0.381493,0.038979,0.331190,-0.058252,0.194288,-0.232693,0.456754,0.541743,0.230925,0.537583,0.860000,NaN
2,NaN,0.800000,2.400000,3.300000,5.900000,1.700000,0.200000,5.900000,7.500000,12.400000,7.700000,-0.300000,2.400000,1.200000,1.300000,2.100000,2.000000,-0.800000,-0.200000,-0.900000,-0.300000,1.000000,1.800000,2.300000,1.600000,0.900000,0.400000,3.100000,0.400000,-0.700000,0.800000,0.500000,2.900000,0.400000,0.600000,2.800000,3.700000,1.500000,2.500000,1.400000,...,0.600000,0.300000,0.600000,1.124764,1.500188,1.200000,0.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.730308,1.027901,0.058279,0.015539,-0.480645,-0.834955,-0.525449,-0.188417,0.735146,0.575450,0.437115,0.649634,0.327930,0.577859,0.277551,0.192262,-0.532763,-0.375475,-0.833657,-0.528164,0.483648,0.565807,0.324510,0.878021,0.639031,0.443672,0.396099,0.945696,0.489083,-1.018483,-0.424372,-0.552050,0.450263,0.614845,0.727277,0.600498,0.305125,0.139133,0.045657,0.106616,...,1.666656,3.042429,2.320857,1.622746,1.594970,0.238658,0.171032,-1.035063,0.111020,0.199216,0.729418,1.102993,0.181123,0.092961,0.014515,0.154629,0.206745,-0.204086,-0.032774,-0.658192,-0.591018,0.863519,0.248491,0.163502,0.581491,0.785682,0.190598,0.563878,0.606804,0.383925,-0.430774,-0.467068,0.054304,-0.319731,0.346839,0.416770,0.463842,0.536374,0.566053,NaN
4,0.415628,1.572848,-1.140994,-0.741962,-0.581395,-0.501253,-0.587741,0.084459,-0.168776,0.084531,0.929054,0.083682,0.250836,0.417014,-0.747508,-0.502092,-0.168209,0.000000,-0.168492,0.000000,0.506329,-0.167926,0.252313,0.838926,1.106530,2.241594,-0.852619,0.000000,0.368550,0.122399,-0.122249,0.122399,0.122249,0.366300,0.243309,0.364078,0.846433,2.158273,-0.586854,0.000000,...,0.480700,0.227500,0.456400,0.233800,0.208000,0.003200,-0.527443,-0.171100,0.109700,0.917200,0.592100,0.413600,0.131500,0.206400,0.504850,0.730264,-0.266128,0.082812,0.542429,0.612361,-0.090600,0.452100,0.593500,0.331600,-0.288400,-0.253700,0.098200,0.801

In [0]:
new_df_gdp.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,-4.611992e+09,-1.850745e+09,-2.945402e+09,-3.258186e+09,-1.744191e+09,-5.382675e+09,-1.341598e+09,-1.740887e+10,-5.015379e+09,-2.948725e+09,-5.782850e+09,-8.701471e+09,-8.416607e+09,-1.303184e+10,-9.866119e+09,NaN,NaN,NaN,NaN
1,-3.914000e+08,-4.081000e+08,-4.149000e+08,-4.244000e+08,-4.351000e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.273300e+11,-2.006800e+11,-1.669000e+11,-2.070800e+11,-2.237500e+11,-2.611600e+11,-3.323400e+11,-2.051200e+11,-3.292300e+11,-3.800000e+11,-8.180700e+11,-7.682400e+11,-1.167660e+12,-1.398840e+12,-1.474300e+12,-1.597800e+12,-1.896000e+12,NaN,NaN
3,7.390000e+11,1.780000e+11,-3.860000e+11,-6.810000e+11,-5.310000e+11,7.870000e+11,5.670000e+11,6.360000e+11,4.060000e+11,1.287000e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-6.669900e+10,-3.893900e+10,-1.383173e+12,-3.045726e+12,-4.910258e+12,-1.800000e+08,3.637930e+10,6.475378e+10,3.376268e+10,9.819843e+10,1.064698e+11,1.319467e+11,-1.580834e+10,-1.375702e+11,-1.975099e+11,NaN,NaN,NaN,NaN


In [0]:
def parse_delta(masks, dir_, seq_len):
    if dir_ == 'backward':
        masks = masks[::-1]

    deltas = []
    # print('masks shape is', masks.shape)

    for h in range(seq_len):
        if h == 0:
            deltas.append(np.ones(1))
        else:
            deltas.append(np.ones(1) + (1 - masks[h]) * deltas[-1])

    return np.array(deltas)

In [0]:
def parse_rec(values, masks, evals, eval_masks, seq_len, dir_):
    deltas = parse_delta(masks, dir_, seq_len)

    rec = {}
    print('2 hat shape', deltas.shape, evals.shape)
    rec['values'] = np.nan_to_num(values).tolist()
    rec['masks'] = masks.astype('int32').tolist()
    # imputation ground-truth
    rec['evals'] = np.nan_to_num(evals).tolist()
    rec['eval_masks'] = eval_masks.astype('int32').tolist()
    #rec['forwards'] = forwards.tolist()
    rec['deltas'] = deltas.tolist()

    return rec

In [0]:
def parse_df(df, index, mean, std, seq_len):

    evals = []

    evals = (df - mean) / std

    evals = evals.reshape(len(evals), 1)

    shp = evals.shape
    # print('shp is', shp)

    evals = evals.reshape(-1)

    # randomly eliminate 10% values as the imputation ground-truth
    indices = np.where(~np.isnan(evals))[0].tolist()
    indices = np.random.choice(indices, len(indices) // 10)

    values = evals.copy()
    values[indices] = np.nan

    masks = ~np.isnan(values)
    eval_masks = (~np.isnan(values)) ^ (~np.isnan(evals))

    evals = evals.reshape(shp)
    values = values.reshape(shp)

    masks = masks.reshape(shp)
    eval_masks = eval_masks.reshape(shp)
    #print('ev', evals.shape, values.shape, masks.shape, eval_masks.shape)
    #label = out.loc[int(id_)]

    #rec = {'label': label}
    
    rec = {}
    
    # prepare the model for both directions
    rec['forward'] = parse_rec(values, masks, evals, eval_masks, seq_len, dir_='forward')
    rec['backward'] = parse_rec(values[::-1], masks[::-1], evals[::-1], eval_masks[::-1], seq_len, dir_='backward')

    rec = json.dumps(rec)

    fs.write(rec + '\n')


    return rec

In [0]:
import json

inflation_dict = {'df': new_df_inflation,
                  'mean': new_df_inflation.stack().mean(),
                  'std': new_df_inflation.stack().std(),
                  'seq_len': len(new_df_inflation.columns),
                  'file_name': 'json_inflation'}

gdp_dict = {'df': new_df_gdp,
            'mean': new_df_gdp.stack().mean(),
            'std': new_df_gdp.stack().std(),
            'seq_len': len(new_df_gdp.columns),
            'file_name': 'json_gdp'}

dict_list = [inflation_dict, gdp_dict]

for dict_item in dict_list:
  fs = open(dict_item['file_name'], 'w')
  
  # for each row of dataframe
  for index, row in dict_item['df'].iterrows():
    test = parse_df(row.values, index, dict_item['mean'], dict_item['std'], dict_item['seq_len'])
  
  fs.close()

2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shape (242, 1) (242, 1)
2 hat shap